In [519]:
import os
from dotenv import load_dotenv
#from openai import OpenAI
from openai import AzureOpenAI
from pydantic import create_model
import inspect, json
from inspect import Parameter
load_dotenv()
print(load_dotenv())
#os.environ['AZURE_OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")
print("API Key:", os.getenv("AZURE_OPENAI_API_KEY2"))



True
API Key: 06f884ba16ec4e97b5be8e4511ee5c93


## **Custom agent**

**Define the functions**

In [520]:
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [521]:
abc(2, 3)

-2

In [522]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [523]:
abc_json = jsonschema(abc)
abc_json

{'name': 'abc',
 'description': 'Compute abc between two numbers',
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `abc`',
  'type': 'object'}}

In [524]:
model_name = "gpt-4o-mini"

**Ask GPT**

In [525]:
client = AzureOpenAI(
    api_key="EnvrvVgQgVNjzHrXY57TUyESTNeIqovna3WsB7mctCSmyrICEJ2lJQQJ99AJACHYHv6XJ3w3AAABACOG2fTJ",
    azure_deployment="gpt35",
    azure_endpoint ="https://aressgenaisvc.openai.azure.com/",
    api_version="2024-06-01"
)
response = client.chat.completions.create(
  model= model_name,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "compute abc between 2 and 3"},
  ],
)

In [526]:
response.choices[0].message.content

'To compute \\(abc\\) between 2 and 3, I would need more information about the variables \\(a\\), \\(b\\), and \\(c\\). Could you please provide the necessary values for \\(a\\), \\(b\\), and \\(c\\)?'

In [527]:
messages= [
    {"role": "user", "content": "Compute abc between 2 and 3"}
]

# Pass the function to GPT model
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    functions=[abc_json],
    function_call="auto",
    temperature=0
)

In [528]:
response

ChatCompletion(id='chatcmpl-AJKAPsLaAXALgrQJHvhKPUZYxYjaq', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "num1": 2,\n  "num2": 3\n}', name='abc'), tool_calls=None), content_filter_results={})], created=1729169117, model='gpt-35-turbo', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=58, total_tokens=82), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])

**Executing the function by extracting the info from the output of the model**

In [529]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.function_call.arguments)
print(type(response.choices[0].message.function_call.arguments))

FunctionCall(arguments='{\n  "num1": 2,\n  "num2": 3\n}', name='abc')
{
  "num1": 2,
  "num2": 3
}
<class 'str'>


In [530]:
func_name = response.choices[0].message.function_call.name
func_args = json.loads(response.choices[0].message.function_call.arguments)
print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

Function name: abc
Function arguments: {'num1': 2, 'num2': 3}
<class 'dict'>


In [531]:
if func_name == 'abc':
    result = abc(**func_args)
print(result)

-2


## **Using Langchain**

In [532]:
from langchain_core.tools import tool

@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [533]:
abc.description

'Compute abc between two numbers'

In [534]:
#from langchain_openai import AzureChatOpenAI
#llm = AzureChatOpenAI(model=os.getenv("AZURE_OPENAI_DEPLOYMENT2"), temperature=0,api_key=os.getenv("AZURE_OPENAI_API_KEY2"),api_version=os.getenv("AZURE_OPENAI_VERSION"))

In [535]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key="EnvrvVgQgVNjzHrXY57TUyESTNeIqovna3WsB7mctCSmyrICEJ2lJQQJ99AJACHYHv6XJ3w3AAABACOG2fTJ",
    azure_deployment="gpt4o",
    model= "gpt-4o",
    azure_endpoint ="https://aressgenaisvc.openai.azure.com/",
    api_version="2024-06-01"
)

In [536]:
tools = [abc]
llm_with_tools = llm.bind_tools(tools)


In [537]:
response = llm_with_tools.invoke("Compute abc between 2 and 3")
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_l7SFFklZVnh6nCKHHn4bCcwx', 'function': {'arguments': '{"num1":2,"num2":3}', 'name': 'abc'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 56, 'total_tokens': 76}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d54531d9eb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}} id='run-94444afe-8c5d-4c65-b2c1-1201bed73b84-0' tool_calls=[{'name': 'abc', 'args': {'num1': 2, 'num2': 3}, 'id': 'call_l7SFFklZVnh6nCKHHn4bCcwx', 'type': 'tool_call'}] usage_metadata={'input_tokens': 56, 'output_tokens': 20, 'total_toke

In [538]:
response.additional_kwargs

{'tool_calls': [{'id': 'call_l7SFFklZVnh6nCKHHn4bCcwx',
   'function': {'arguments': '{"num1":2,"num2":3}', 'name': 'abc'},
   'type': 'function'}]}